Thanks to the starter code - https://huggingface.co/spaces/PyaeSoneK/chatchat


In [1]:
!nvidia-smi

Thu Sep 14 05:03:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install the libraries

In [2]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00


Load the libraries

In [3]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap


Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [4]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [5]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Define Transformers Pipeline which will be fed into Langchain

In [6]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [16]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
As the leader of a sizable team in a dynamic business, I'm tasked with improving our supply chain management process. Recently, we've been facing issues like increased costs, longer lead times, and decreased customer satisfaction, all of which we believe are interconnected. To address these challenges, I need your assistance in optimizing our supply chain management. Please provide insights, strategies, and best practices that can help us streamline our operations, reduce costs, improve efficiency, and ultimately enhance customer satisfaction. Additionally, consider the latest technologies and innovations that could be integrated into our supply chain to make it more agile and responsive to market demands.If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


All the helper fucntions to generate prompt, prompt template, clean up output text

In [17]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [18]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.4,'max_length': 256, 'top_k' :50})



In [19]:
system_prompt = "You are an advanced supply chain optimization expert"
instruction = "Use the data provided to you to optimize the supply chain:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are an advanced supply chain optimization expert
<>

Use the data provided to you to optimize the supply chain:

 {text}[/INST]


In [20]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)



In [21]:
text = "Based on the data provided how can I optimize my supply chain? Product,Supplier,Unit Cost,Lead Time (days),Demand (units) Product A,Supplier 1,10,7,100 Product B,Supplier 2,15,10,150 Product C,Supplier 3,20,14,120 Product D,Supplier 1,12,7,80 Product E,Supplier 2,18,10,90 Product F,Supplier 3,22,14,110 Product G,Supplier 1,8,7,70 Product H,Supplier 2,14,10,100 Product I,Supplier 3,18,14,80"

In [22]:
response = llm_chain.run(text)
print(response)

  As an advanced supply chain optimization expert, I would use the following techniques to optimize your supply chain based on the data provided:

1. Identify the most critical products:
a. Calculate the total cost of ownership (TCO) for each product, including unit cost, lead time, and demand.
b. Analyze the TCO for each product to identify the most critical products that are high-value, high-demand, and low-lead-time products.
c. Focus on optimizing the supply chain for these critical products to reduce costs and improve service levels.
2. Conduct a comprehensive supplier analysis:
a. Evaluate each supplier based on factors such as quality, reliability, lead time, and flexibility.
b. Identify the top-performing suppliers and allocate more business to them.
c. Consider diversifying the supplier base to reduce risk and improve supply chain resilience.
3. Optimize inventory levels:
a. Develop an inventory optimization model that takes into account the demand forecast, lead times, and un